# Plot contour line Y band resolution vs (dPWV,PWV) and Loop on airmass

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/11/18
- last update : 2025/11/18
- last update : 2025/11/19 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.cm as cm
from matplotlib.lines import Line2D


from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd
import h5py

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
from scipy.interpolate import splprep, splev

In [ ]:
# where are stored the figures
pathfigs = "figs_LOOPairmassPlotContourLineszdpwvzpwv"
prefix = "pwv-resoreq-am"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

## Functions

In [ ]:
def resample_curve(x, y, spacing):
    """Return a new curve with points spaced approximately 'spacing' apart."""
    # parametrize original curve
    tck, u = splprep([x, y], s=0)
    
    # coordinates length
    pts = np.vstack([x, y]).T
    d = np.sqrt(((pts[1:] - pts[:-1])**2).sum(axis=1))
    s_total = d.sum()
    
    # new parameterization
    u_new = np.linspace(0, 1, int(s_total/spacing))
    x_new, y_new = splev(u_new, tck)
    return np.array([x_new, y_new]).T

## Configuration

In [ ]:
!ls data/mag_vs_zdpwvzpwv 

In [ ]:
data_path = "data/mag_vs_zdpwvzpwv/results"

In [ ]:
all_airmass = np.array([1.0,1.2,1.5,2.0]) 

In [ ]:
Delta_PWV = 0.5
Delta_PWV_num = int(Delta_PWV*100.)
#all_PWV = np.arange(Delta_PWV,20.,Delta_PWV)
#print("PWV values",all_PWV) 

In [ ]:
#all_DPWV = np.arange(2.,0.,-0.01)
Delta_dPWV = 0.02
Delta_dPWV_num = int(Delta_dPWV*1000.)
#all_DPWV = np.arange(Delta_dPWV,1.,Delta_dPWV)
#print(all_DPWV)

## Read files

In [ ]:
N = len(all_airmass)

In [ ]:
all_data = []

for idx_am in range(N):
    am = all_airmass[idx_am] 
    am_num = int(am*10.)

    filename_input =  f"MagResolutionFromPWVvsPWV_am{am_num}_DPWV{Delta_PWV_num}_DdPWV{Delta_dPWV_num}.h5"
    fullfilename_input = os.path.join(data_path,filename_input)

    with h5py.File(fullfilename_input, "r") as f:
        print(list(f.keys()))

    with h5py.File(fullfilename_input, "r") as f:
        zPWV   = f["zPWV"][:]
        zdPWV  = f["zdPWV"][:]
        tresY  = f["tresY"][:]
        tresZY = f["tresZY"][:]

    dict_amtodata = {"zPWV":zPWV, "zdPWV":zdPWV,"tresY":tresY,"tresZY":tresZY}
    all_data.append(dict_amtodata)
    

## Extract contour levels

In [ ]:
levels = np.array([0.5,1.,5.,10.,15.,20.,25.,30.])

In [ ]:
# Grille 2x2 + une colonne étroite pour la colorbar
fig = plt.figure(figsize=(16, 14))
gs = fig.add_gridspec(2, 3, width_ratios=[1, 1, 0.05], wspace=0.3, hspace=0.3)

axes = [
    fig.add_subplot(gs[0, 0]),
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[1, 0]),
    fig.add_subplot(gs[1, 1]),
]

all_cs = []

for idx_am in range(N):
    am = all_airmass[idx_am]
    dict_amtodata = all_data[idx_am]

    zPWV  = dict_amtodata["zPWV"]
    zdPWV = dict_amtodata["zdPWV"]
    tresY = dict_amtodata["tresY"]

    x = zPWV
    y = zdPWV
    Z = tresY

    X, Y = np.meshgrid(x, y)

    ax = axes[idx_am]

    pcm = ax.pcolormesh(X, Y, Z, shading='auto', cmap='magma')

    cs = ax.contour(X, Y, Z, levels=levels, colors='w', linewidths=1.0)
    ax.clabel(cs, inline=True, fontsize=8)

    ax.set_xlabel('airmass × PWV (mm)')
    ax.set_ylabel('airmass × ΔPWV (mm)')
    ax.set_title(f'Y band – airmass = {am}')

    ax.minorticks_on()
    all_cs.append(cs)

# Colorbar dans la dernière colonne
cax = fig.add_subplot(gs[:, 2])
cbar = fig.colorbar(pcm, cax=cax, label='$\\sigma(m_y)$ (mmag)')

figname =f"{pathfigs}/{prefix}_amdeltapwv_vs_amPWV_linvscale"+figtype
plt.savefig(figname)

plt.show()


In [ ]:
# -------------------------
#  Grille 2×2 + colonne colorbar
# -------------------------
fig = plt.figure(figsize=(16, 14))
gs = fig.add_gridspec(2, 3, width_ratios=[1, 1, 0.05], wspace=0.3, hspace=0.3)

axes = [
    fig.add_subplot(gs[0, 0]),
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[1, 0]),
    fig.add_subplot(gs[1, 1]),
]

# -----------------------------------
#  Dessin de chaque subplot
# -----------------------------------

for idx_am in range(4):   # N = 4
    ax = axes[idx_am]

    am = all_airmass[idx_am]
    d = all_data[idx_am]

    x = d["zPWV"]
    y = d["zdPWV"]
    Z = d["tresY"]

    X, Y = np.meshgrid(x, y)

    # Charger les contours déjà calculés
    cs = all_cs[idx_am]

    # -----------------------------
    #    IMAGE DE FOND
    # -----------------------------
    ax.imshow(Z, origin='lower',
              extent=[x.min(), x.max(), y.min(), y.max()],
              aspect='auto', cmap='magma', alpha=0.10)

    # -----------------------------
    #   COLORATION MANUELLE DES CONTOURS
    # -----------------------------
    levels = cs.levels
    cmap = plt.cm.coolwarm
    norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

    for level, segs in zip(cs.levels, cs.allsegs):
        col = cmap(norm(level))
        for curve in segs:
            ax.plot(curve[:, 0], curve[:, 1],
                    '-', lw=2.0, color=col)

            # --- Label numérique du niveau ---
            mid = len(curve) // 2
            xm, ym = curve[mid]
            ax.text(xm, ym, f"{level:g}",
                color=col, fontsize=7, fontweight="bold",
                ha="center", va="center",
                bbox=dict(facecolor="white", edgecolor="none", alpha=0.5))

    # Étiquettes
    ax.set_xlabel('airmass × PWV (mm)')
    ax.set_ylabel('airmass × ΔPWV (mm)')
    ax.set_title(f'Magnitude resolution (Y band, airmass = {am})')
    ax.minorticks_on()

# ------------------------------------------------
#   Colorbar basée sur les niveaux (colormap)
# ------------------------------------------------

# Petite astuce : créer une ScalarMappable juste pour la colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cax = fig.add_subplot(gs[:, 2])
cbar = fig.colorbar(sm, cax=cax, label="Contour level")

plt.tight_layout()

figname =f"{pathfigs}/{prefix}_ContourLines_amdeltapwv_vs_amPWV_linvscale"+figtype
plt.savefig(figname)



plt.show()


In [ ]:
fig = plt.figure(figsize=(16, 13), constrained_layout=True)  # <<-- important
gs = fig.add_gridspec(2, 3, width_ratios=[1, 1, 0.05])

axes = [
    fig.add_subplot(gs[0, 0]),
    fig.add_subplot(gs[0, 1], sharey=None),
    fig.add_subplot(gs[1, 0], sharey=None),
    fig.add_subplot(gs[1, 1], sharey=None),
]

# Partage des axes Y
axes[1].sharey(axes[0])
axes[2].sharey(axes[0])
axes[3].sharey(axes[0])

# Boucle pour tracer tes subplots
for idx_am in range(4):
    ax = axes[idx_am]

    am = all_airmass[idx_am]
    d = all_data[idx_am]

    x = d["zPWV"]
    y = d["zdPWV"]
    Z = d["tresY"]

    X, Y = np.meshgrid(x, y)

    cs = all_cs[idx_am]
    levels = cs.levels
    cmap = plt.cm.coolwarm
    norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

    # Contours colorés
    for level, segs in zip(cs.levels, cs.allsegs):
        col = cmap(norm(level))
        for curve in segs:
            ax.plot(curve[:, 0], curve[:, 1]/level, '-', lw=2.0, color=col)

    ax.set_xlabel('airmass × PWV (mm)')
    if idx_am in [0, 2]:
        ax.set_ylabel('airmass × ΔPWV (mm)')
    else:
        ax.set_ylabel("")
        ax.tick_params(labelleft=False)

    ax.set_title(f'Magnitude resolution (Y band, airmass = {am})', fontsize=13)
    ax.minorticks_on()

    ax.grid()

# Colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cax = fig.add_subplot(gs[:, 2])
cbar = fig.colorbar(sm, cax=cax, label="Contour level")

figname =f"{pathfigs}/{prefix}_ContourLinesNormalized_amdeltapwv_vs_amPWV_linvscale"+figtype
plt.savefig(figname)


plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D

fig, ax = plt.subplots(figsize=(10, 9))

# Choix d'une colormap pour les airmass
cmap_am = plt.cm.coolwarm
norm_am = mcolors.Normalize(vmin=min(all_airmass), vmax=max(all_airmass))

# Boucle sur les airmass
for idx_am, am in enumerate(all_airmass):
    cs = all_cs[idx_am]      # contours déjà calculés

    color = cmap_am(norm_am(am))   # couleur dépendant uniquement de l'airmass

    # tracer tous les segments de tous les niveaux
    for level, segs in zip(cs.levels, cs.allsegs):
        for curve in segs:
            ax.plot(curve[:, 0], curve[:, 1]/level, '-', lw=1.5, color=color)

# Labels et axes
ax.set_xlabel('airmass × PWV (mm)')
ax.set_ylabel('airmass × ΔPWV (mm)')
ax.set_title('Contours superposés pour toutes les airmasses')
ax.minorticks_on()
ax.grid(True)

# Légende : 1 ligne par airmass
legend_elements = [
    Line2D([0], [0], color=cmap_am(norm_am(am)), lw=2, label=f'airmass = {am}')
    for am in all_airmass
]
ax.legend(handles=legend_elements, loc='upper right', title="Airmass")

figname =f"{pathfigs}/{prefix}_ContourLinesNormalizedSuperipposed_amdeltapwv_vs_amPWV_linvscale"+figtype
plt.savefig(figname)


plt.show()


## Functions to Fit

$F \simeq e^{-K_Y \cdot X^\alpha}$

$m = \frac{2.5}{\ln(10)} K_Y \cdot X^\alpha $

- $X = z \cdot PWV$

$$ dX = \frac{1}{\alpha} \frac{\ln(10)}{2.5} \frac{dm_Y}{K_Y}\cdot X^{(1-\alpha)} = \beta \cdot dm_Y \cdot X^{1-\alpha}$$

$$\frac{dX}{X} = \frac{\ln(10)}{2.5} \frac{1}{\alpha K_Y} \frac{dm_Y}{X^\alpha} = \beta \cdot \frac{dm_Y}{X^\alpha}$$

$$K_Y =  \frac{\ln(10)}{2.5}\frac{1}{\alpha \beta}$$

In [ ]:
def func01(x,alpha,beta):
    """
    """
    return np.power(x,1-alpha)*beta

In [ ]:
def func1(x,alpha,beta,dm):
    """
    """
    return np.power(x,1-alpha)*beta*dm

In [ ]:
def func02(x,alpha,beta):
    """
    """
    return np.power(x,-alpha)*beta

In [ ]:
def func2(x,alpha,beta,dm):
    """
    """
    return np.power(x,-alpha)*beta*dm

#### Gather datapoints

In [ ]:
# Boucle sur les airmass
all_Xpoints = []
all_Ypoints = []
for idx_am, am in enumerate(all_airmass):
    cs = all_cs[idx_am]      # contours déjà calculés

    color = cmap_am(norm_am(am))   # couleur dépendant uniquement de l'airmass

    # tracer tous les segments de tous les niveaux
    for level, segs in zip(cs.levels, cs.allsegs):
        for curve in segs:
            Xpoints = curve[:, 0]
            Ypoints = curve[:, 1]/level
            all_Xpoints.append(Xpoints)
            all_Ypoints.append(Ypoints)
               

In [ ]:
Xpoints = np.concat(all_Xpoints)

In [ ]:
Ypoints = np.concat(all_Ypoints)

In [ ]:
plt.plot(Xpoints,Ypoints,'.')

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
xfit = np.linspace(Xpoints.min(),Xpoints.max(),100)

In [ ]:
dm=1
def func_to_fit(x,alpha,beta):
    return func1(x,alpha,beta,dm=dm)
alpha0 = 0.5
beta0 = 0.1

In [ ]:
popt, pcov = curve_fit(func_to_fit, Xpoints, Ypoints, p0=[alpha0, beta0])
yfit = func_to_fit(xfit,*popt)
# Erreurs sur les paramètres
perr = np.sqrt(np.diag(pcov))

ax.plot(xfit,yfit,"-",lw=3,color=color)

# Stocker les résultats du fit pour la légende
fit_results = {
        "alpha": popt[0],
        "beta": popt[1],
}

# Texte à afficher
fit_text = (
    "fit results : \n"
    f"$\\alpha = {popt[0]:.3f} \\pm {perr[0]:.3f}$\n"
    f"$\\beta = {popt[1]:.3f} \\pm {perr[1]:.3f}$"
)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), layout="constrained")
ax.plot(Xpoints,Ypoints,'.',c="b")
ax.plot(xfit,yfit,"-",lw=3,color="r")
ax.set_xlabel("$airmass \\cdot PWV$ (mm)", fontsize=13)
ax.set_ylabel("$airmass \\cdot \\sigma$ PWV (mm)", fontsize=13)
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)", fontsize=15)
ax.text(
    0.1, 0.98, fit_text,
    transform=ax.transAxes,
    fontsize=18,
    verticalalignment='top',color="g",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8)
)


In [ ]:
alpha = popt[0]
beta = popt[1]
K= 1/alpha/beta*np.log(10)/2.5

In [ ]:
K